<a href="https://colab.research.google.com/github/caroalvarezf/Trabajo-Especiliazacion/blob/master/01_PREPARACION_DATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/caroalvarezf/Trabajo-Especiliazacion/master/init.py
import init; init.init(force_download=False) 

replicating local resources


In [2]:
import pandas as pd
datos=pd.read_csv("local/DATA_DEMO.csv", sep=";")

In [3]:
datos.head()

,CODIGO_CLIENTE,FECHA_CONTACTO,CANAL,COD_CANAL,COD_SERVICIO,sexo,Fecha de nacimiento,segmento,producto1,producto2,producto3
0,1,19/10/2019,OFICINAS,1,5,m,8/01/1974,rentas masivas,si,si,no
1,1,3/06/2020,OFICINAS,1,5,m,8/01/1974,rentas masivas,si,si,no
2,1,2/04/2019,OFICINAS,1,5,m,8/01/1974,rentas masivas,si,si,no
3,1,24/04/2019,OFICINAS,1,5,m,8/01/1974,rentas masivas,si,si,no
4,1,1/04/2020,OFICINAS,1,5,m,8/01/1974,rentas masivas,si,si,no


In [ ]:
!git init

Initialized empty Git repository in /content/.git/
